### AutoGen

A framework for building AI agents and applications

https://microsoft.github.io/autogen/stable/index.html

In [1]:
# !pip install -U "autogen-agentchat" "autogen-ext[openai]"
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
agent = AssistantAgent("assistant", OpenAIChatCompletionClient(model="gpt-4o-mini"))
await agent.run(task="Say 'Hello World!'")

TaskResult(messages=[TextMessage(source='user', models_usage=None, content="Say 'Hello World!'", type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=41, completion_tokens=3), content='Hello World!', type='TextMessage')], stop_reason=None)

### Quickstart

In [4]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient


In [5]:
# Define a tool
async def get_weather(city: str) -> str:
    return f"The weather in {city} is 73 degrees and Sunny."

# Define an agent
weather_agent = AssistantAgent(
    name="weather_agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    tools=[get_weather],
)

# Define a team with a single agent and maximum auto-gen turns of 1.
agent_team = RoundRobinGroupChat([weather_agent], max_turns=1)

while True:
    # Get user input from the console.
    user_input = input("Enter a message (type 'exit' to leave): ")
    if user_input.strip().lower() == "exit":
        break
    # Run the team and stream messages to the console.
    stream = agent_team.run_stream(task=user_input)
    await Console(stream)

---------- user ----------
hangzhou
---------- weather_agent ----------
[FunctionCall(id='call_T4UV6X7aSCzFA5TSPF4cCB4A', arguments='{"city":"Hangzhou"}', name='get_weather')]
---------- weather_agent ----------
[FunctionExecutionResult(content='The weather in Hangzhou is 73 degrees and Sunny.', call_id='call_T4UV6X7aSCzFA5TSPF4cCB4A')]
---------- weather_agent ----------
The weather in Hangzhou is 73 degrees and Sunny.


### Models

In [6]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import UserMessage

openai_model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
result = await openai_model_client.create([UserMessage(content="What is the capital of France?", source="user")])

print(result)

finish_reason='unknown' content='The capital of France is Paris.' usage=RequestUsage(prompt_tokens=15, completion_tokens=7) cached=False logprobs=None


### Message

Types of Messages

At a high level, messages in AgentChat can be categorized into two types: agent-agent messages and an agent’s internal events and messages.

In [7]:
from autogen_agentchat.messages import TextMessage

text_message = TextMessage(content="Hello, world!", source="User")

text_message

TextMessage(source='User', models_usage=None, content='Hello, world!', type='TextMessage')

In [42]:
from io import BytesIO

import requests
from autogen_agentchat.messages import MultiModalMessage
from autogen_core import Image as AGImage
from PIL import Image

pil_image = Image.open(BytesIO(requests.get("https://picsum.photos/300/200").content))
img = AGImage(pil_image)
multi_modal_message = MultiModalMessage(content=["Can you describe the content of this image?", img], source="User")

multi_modal_message

MultiModalMessage(source='User', models_usage=None, content=['Can you describe the content of this image?', <autogen_core._image.Image object at 0x1166120d0>], type='MultiModalMessage')

### Agents

In [7]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient


# Define a tool that searches the web for information.
async def web_search(query: str) -> str:
    """Find information on the web"""
    return "AutoGen is a programming framework for building multi-agent applications."


# Create an agent that uses the OpenAI GPT-4o model.
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[web_search],
    system_message="Use tools to solve tasks.",
)

In [8]:
response = await agent.on_messages(
    [TextMessage(content="Find information on AutoGen", source="user")],
    cancellation_token=CancellationToken(),
)

print("Inner Messages:")
print(response.inner_messages)
print("Chat Message:")
print(response.chat_message)


Inner Messages:
[ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=61, completion_tokens=15), content=[FunctionCall(id='call_obMFYHxfryeKriEf2o7HeW2O', arguments='{"query":"AutoGen"}', name='web_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, content=[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', call_id='call_obMFYHxfryeKriEf2o7HeW2O')], type='ToolCallExecutionEvent')]
Chat Message:
source='assistant' models_usage=None content='AutoGen is a programming framework for building multi-agent applications.' type='ToolCallSummaryMessage'


In [9]:
response = await Console(
    agent.on_messages_stream(
        [TextMessage(content="Find information on AutoGen", source="user")],
        cancellation_token=CancellationToken(),
    )
)

---------- assistant ----------
[FunctionCall(id='call_jtN93hIzVEuvAJi3XqlePCR0', arguments='{"query":"AutoGen software"}', name='web_search')]
---------- assistant ----------
[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', call_id='call_jtN93hIzVEuvAJi3XqlePCR0')]
---------- assistant ----------
AutoGen is a programming framework for building multi-agent applications.


In [10]:
async for response in agent.on_messages_stream(
    [TextMessage(content="Find information on AutoGen", source="user")],
    cancellation_token=CancellationToken(),
):
    print(response)

source='assistant' models_usage=RequestUsage(prompt_tokens=156, completion_tokens=19) content=[FunctionCall(id='call_eckUMTkuoq9sx1HNkcdlD3Bs', arguments='{"query":"AutoGen project or software overview"}', name='web_search')] type='ToolCallRequestEvent'
source='assistant' models_usage=None content=[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', call_id='call_eckUMTkuoq9sx1HNkcdlD3Bs')] type='ToolCallExecutionEvent'
Response(chat_message=ToolCallSummaryMessage(source='assistant', models_usage=None, content='AutoGen is a programming framework for building multi-agent applications.', type='ToolCallSummaryMessage'), inner_messages=[ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=156, completion_tokens=19), content=[FunctionCall(id='call_eckUMTkuoq9sx1HNkcdlD3Bs', arguments='{"query":"AutoGen project or software overview"}', name='web_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent

In [11]:
import pandas as pd
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_experimental.tools.python.tool import PythonAstREPLTool

df = pd.read_csv("https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv")
tool = LangChainToolAdapter(PythonAstREPLTool(locals={"df": df}))
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
agent = AssistantAgent(
    "assistant", tools=[tool], model_client=model_client, 
    system_message=f"Use the `df` variable to access the dataset,column names are {df.columns}.",
    reflect_on_tool_use=False
)

await Console(
    agent.on_messages_stream(
        [TextMessage(content="What's the average age of the passengers?", source="user")], CancellationToken()
    )
)

await Console(
    agent.on_messages_stream(
        [TextMessage(content="What is the average age of survivors? Please display the results for all genders as well as separately for each gender.", source="user")], CancellationToken()
    )
)

---------- assistant ----------
[FunctionCall(id='call_PvZFmJm3aH0rEJ7pzOLlmCH0', arguments='{"query":"df[\'Age\'].mean()"}', name='python_repl_ast')]
---------- assistant ----------
[FunctionExecutionResult(content='29.69911764705882', call_id='call_PvZFmJm3aH0rEJ7pzOLlmCH0')]
---------- assistant ----------
29.69911764705882


Response(chat_message=ToolCallSummaryMessage(source='assistant', models_usage=None, content='28.343689655172415\nSex\nfemale    28.847716\nmale      27.276022\nName: Age, dtype: float64', type='ToolCallSummaryMessage'), inner_messages=[ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=234, completion_tokens=81), content=[FunctionCall(id='call_MsotkfjQbzu3LpLimzNjiZtZ', arguments='{"query": "df[df[\'Survived\'] == 1][\'Age\'].mean()"}', name='python_repl_ast'), FunctionCall(id='call_4eTgTz3oihVwpNVLLCJWB74O', arguments='{"query": "df[df[\'Survived\'] == 1].groupby(\'Sex\')[\'Age\'].mean()"}', name='python_repl_ast')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, content=[FunctionExecutionResult(content='28.343689655172415', call_id='call_MsotkfjQbzu3LpLimzNjiZtZ'), FunctionExecutionResult(content='Sex\nfemale    28.847716\nmale      27.276022\nName: Age, dtype: float64', call_id='call_4eTgTz3oihVwpNVLLCJWB74O'

---------- assistant ----------
[FunctionCall(id='call_MsotkfjQbzu3LpLimzNjiZtZ', arguments='{"query": "df[df[\'Survived\'] == 1][\'Age\'].mean()"}', name='python_repl_ast'), FunctionCall(id='call_4eTgTz3oihVwpNVLLCJWB74O', arguments='{"query": "df[df[\'Survived\'] == 1].groupby(\'Sex\')[\'Age\'].mean()"}', name='python_repl_ast')]


By default, `AssistantAgent` uses the `UnboundedChatCompletionContext` which sends the full conversation history to the model.

To limit the context to the last n messages, you can use the `BufferedChatCompletionContext`.

In [5]:
from autogen_core.model_context import BufferedChatCompletionContext
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken

# Define a tool that searches the web for information.
async def web_search(query: str) -> str:
    """Find information on the web"""
    return "AutoGen is a programming framework for building multi-agent applications."


model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

# Create an agent that uses only the last 5 messages in the context to generate responses.
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[web_search],
    system_message="Use tools to solve tasks.",
    model_context=BufferedChatCompletionContext(buffer_size=5),  # Only use the last 5 messages in the context.
)

response = await agent.on_messages(
    [TextMessage(content="Find information on AutoGen", source="user")],
    cancellation_token=CancellationToken(),
)

print("Inner Messages:")
print(response.inner_messages)
print("Chat Message:")
print(response.chat_message)

Inner Messages:
[ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=61, completion_tokens=15), content=[FunctionCall(id='call_DFCw6ETBs5lSVrPdxbczGnZf', arguments='{"query":"AutoGen"}', name='web_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, content=[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', call_id='call_DFCw6ETBs5lSVrPdxbczGnZf')], type='ToolCallExecutionEvent')]
Chat Message:
source='assistant' models_usage=None content='AutoGen is a programming framework for building multi-agent applications.' type='ToolCallSummaryMessage'


### Team

In [1]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an OpenAI model client.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
)

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)

In [2]:
result = await team.run(task="Write a short poem about the fall season.")
print(result)

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=107), content='In amber hues the leaves descend,  \nA gentle dance, the seasons blend,  \nCrisp whispers in the cooling air,  \nNature’s quilt, a soft repair.  \n\nPumpkins shine on doorsteps bright,  \nWhile harvest moons bring cozy nights,  \nThe rustling trees, a soothing sound,  \nAs autumn’s magic wraps the ground.  \n\nSweaters on, we gather near,  \nWith laughter shared and warmth sincere,  \nIn every swirl of gold and brown,  \nThe beauty of fall brings joy renowned.  ', type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=153, completion_tokens=241), content='This poem beautifully captures the essence of the fall season. The imagery of amber hues and cozy nights evokes a strong sense of warmth and nostalgia. Her

In [7]:
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None content='Write a short poem about the fall season.' type='TextMessage'
source='primary' models_usage=RequestUsage(prompt_tokens=28, completion_tokens=125) content="In autumn's glow, the leaves cascade,  \nA tapestry of golds and reds displayed.  \nCrisp air whispers secrets from the trees,  \nAs acorns tumble, carried by the breeze.  \n\nPumpkins glow beneath the harvest moon,  \nWhile cinnamon scents fill each cozy room.  \nNature's palette, vibrant, rich, and bright,  \nAs days grow shorter, kissed by fading light.  \n\nOh, fall, with your charm, so bittersweet,  \nA time for gathering, where hearts can meet.  \nIn every rustle, in every sigh,  \nThe magic of the season bids goodbye." type='TextMessage'
source='critic' models_usage=RequestUsage(prompt_tokens=171, completion_tokens=234) content='Your poem beautifully captures the essence of fall with vivid imagery and a rhythmic flow. The use of sensory details, such as "crisp air" and "cinnamon scents,

In [10]:
await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a short poem about the fall season."))  # Stream the messages to the console.

---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
In autumn's glow, the leaves descend,  
A tapestry of gold and red,  
The crisp air whispers, summers end,  
As nature dons her vibrant bed.  

With pumpkins bright and harvest cheer,  
The fields are ripe, the skies are clear,  
A fleeting dance, the world in tune,  
Embracing change, beneath the moon.  

So gather 'round as days grow short,  
With cider warm and laughter's sport,  
In every shade, a story spun,  
The fall's sweet breath, our hearts have won.
---------- critic ----------
Your poem beautifully captures the essence of autumn with vivid imagery and a rhythmic flow. I appreciate how you evoke the sensory experiences of the season, from the visual beauty of the leaves to the warmth of cider. 

To enhance your poem further, consider the following feedback:

1. **Imagery**: While you have great imagery, adding a few more specific details could enrich the visual experience. Perh

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=113), content="In autumn's glow, the leaves descend,  \nA tapestry of gold and red,  \nThe crisp air whispers, summers end,  \nAs nature dons her vibrant bed.  \n\nWith pumpkins bright and harvest cheer,  \nThe fields are ripe, the skies are clear,  \nA fleeting dance, the world in tune,  \nEmbracing change, beneath the moon.  \n\nSo gather 'round as days grow short,  \nWith cider warm and laughter's sport,  \nIn every shade, a story spun,  \nThe fall's sweet breath, our hearts have won.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=159, completion_tokens=189), content='Your poem beautifully captures the essence of autumn with vivid imagery and a rhythmic flow. I appreciate how you evoke the sensory experiences 

In [12]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create the agents.
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
assistant = AssistantAgent("assistant", model_client=model_client)
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

# Create the termination condition which will end the conversation when the user says "APPROVE".
termination = TextMentionTermination("APPROVE")

# Create the team.
team = RoundRobinGroupChat([assistant, user_proxy], termination_condition=termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="Write a 4-line poem about the ocean.")
# Use asyncio.run(...) when running in a script.
await Console(stream)

---------- user ----------
Write a 4-line poem about the ocean.
---------- assistant ----------
Waves dance lightly on the shore,  
Whispers of secrets, legends of yore.  
Endless blue where horizons blend,  
In the ocean's embrace, our spirits mend.  
TERMINATE
---------- user_proxy ----------
add some star
---------- assistant ----------
Stars twinkle bright in the evening sky,  
As waves dance lightly, whispering by.  
Endless blue where horizons blend,  
In the ocean's embrace, our spirits mend.  
TERMINATE
---------- user_proxy ----------
APPROVE


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a 4-line poem about the ocean.', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=46, completion_tokens=42), content="Waves dance lightly on the shore,  \nWhispers of secrets, legends of yore.  \nEndless blue where horizons blend,  \nIn the ocean's embrace, our spirits mend.  \nTERMINATE", type='TextMessage'), UserInputRequestedEvent(source='user_proxy', models_usage=None, request_id='98ac9fba-659b-46d9-8d67-ac33cc721b0a', content='', type='UserInputRequestedEvent'), TextMessage(source='user_proxy', models_usage=None, content='add some star', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=103, completion_tokens=42), content="Stars twinkle bright in the evening sky,  \nAs waves dance lightly, whispering by.  \nEndless blue where horizons blend,  \nIn the ocean's embrace, our spirits mend.  \nTERMINATE", type='TextMess